# 应用深度学习构建深层特征的图片检索系统

## 导入Graphlab Create

In [2]:
import graphlab

## 导入CIFAR-10数据集

In [4]:
image_train = graphlab.SFrame("ML_Data/image_train_data/")

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\lenovo\AppData\Local\Temp\graphlab_server_1517642454.log.0


This non-commercial license of GraphLab Create for academic use is assigned to donghui_ncepu@126.com and will expire on January 25, 2019.


In [5]:
deep_learning_model = graphlab.load_model("http://s3.amazonaws.com/GraphLaob-Datasets/deeplearning/imagenet_model_iter45")
deep_learning_model.extract_features(image_train)

Downloading http://s3.amazonaws.com/GraphLaob-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini to C:/Users/lenovo/AppData/Local/Temp/graphlab-lenovo/21000/03b9dd3f-f69d-4e49-a567-7d0fb1970efc.ini

Failed to download http://s3.amazonaws.com/GraphLaob-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini: HTTP response code said error

IOError: Unable to load model from http://s3.amazonaws.com/GraphLaob-Datasets/deeplearning/imagenet_model_iter45: Cannot open http://s3.amazonaws.com/GraphLaob-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini for read. Fail to download from http://s3.amazonaws.com/GraphLaob-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini. HTTP response code said error

In [6]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


## 通过这些深度特征模型进行最近邻模型

In [7]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=["deep_features"],label="id")

Starting brute force nearest neighbors model training.

## 通过应用了深度特征的图像检索模型找到相似图片

In [8]:
graphlab.canvas.set_target("ipynb")

In [9]:
cat = image_train[18:19]

In [10]:
cat["image"].show()

In [11]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 25.568ms     |

| Done         |         | 100         | 529.408ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


In [12]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result["reference_label"],"id")

In [13]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.039ms     |

| Done         |         | 100         | 511.861ms    |

+--------------+---------+-------------+--------------+

In [14]:
cat_neighbors["image"].show()

## 检索和轿车相似的图像

In [15]:
car = image_train[8:9]

In [16]:
car["image"].show()

In [17]:
get_images_from_ids(knn_model.query(car))["image"].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.541ms     |

| Done         |         | 100         | 594.08ms     |

+--------------+---------+-------------+--------------+

## 应用lambda函数进行图像检索

In [29]:
show_images = lambda i : get_images_from_ids(knn_model.query(image_train[i:i+1]))["image"].show()

In [30]:
show_images(34)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 14.537ms     |

| Done         |         | 100         | 477.771ms    |

+--------------+---------+-------------+--------------+

In [31]:
show_images(123)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.627ms     |

| Done         |         | 100         | 421.927ms    |

+--------------+---------+-------------+--------------+